In [ ]:
import pandas as pd
from SharekhanApi.sharekhanConnect import SharekhanConnect

api_key = "GIaiFf8eFa1gOfdC2W4m6RUb9YOvVYIu"
with open('access.txt', 'r') as file:
    access_token = file.read().strip()
    
sharekhan = SharekhanConnect(api_key, access_token=access_token)

exchange = "NC"
scripcode="1333"
interval="60minute"
order = sharekhan.historicaldata(exchange, scripcode, interval)

if order:
    frame = order['data']
    hist_df = pd.DataFrame(frame)
# Step 1: Convert to datetime with explicit format
hist_df['tradeDate'] = pd.to_datetime(hist_df['tradeDate'], format="%d/%m/%Y", errors='coerce')
filtered_data = hist_df[hist_df['tradeDate'] == pd.to_datetime("2025-09-04")]
print("\n📊 Historical Data:")
print(filtered_data)


📊 Historical Data:
       open     high      low    close      qty tradeTime  tradeDate
49  1006.45  1009.25   998.40   999.35   950220  10:14:59 2025-08-04
50   999.35  1002.60   999.35  1001.25  1085839  11:14:59 2025-08-04
51  1001.25  1002.50   999.95  1000.90   588825  12:14:58 2025-08-04
52  1000.90  1001.85  1000.00  1000.25   531953  13:14:58 2025-08-04
53  1000.30  1000.40   995.00   995.35  1287486  14:14:58 2025-08-04
54   995.30   996.45   994.05   995.70  1802060  15:14:59 2025-08-04
55   995.65   997.40   995.55   996.00   601253  15:29:59 2025-08-04


In [ ]:
import pandas as pd
from SharekhanApi.sharekhanConnect import SharekhanConnect

api_key = "GIaiFf8eFa1gOfdC2W4m6RUb9YOvVYIu"
with open('access.txt','r')as file:
    access_token = file.read().strip()
sharekhan = SharekhanConnect(api_key, access_token=access_token)

exchange = "MX"

try:
    order = sharekhan.master(exchange)
    if order:
        print("Data fetched successfully.")
    else:
        print("No data received. Check exchange code or credentials.")
except Exception as e:
    print("Error occurred:", e)
    order = None

if order:
    frame = order['data']
    df = pd.DataFrame(frame)

    # Filter for 'GOLDM' or 'SILVERM' in tradingSymbol AND optionType = FUT
    filtered_df = df[
        df['tradingSymbol'].str.contains('GOLDM|SILVERM', case=False, na=False) &
        (df['optionType'] == 'FUT')
    ]
    cleaned_df = filtered_df.dropna(axis=1, how='all').drop(columns=['strike','instType','optionType'], errors='ignore')
    print(cleaned_df)

   


Data fetched successfully.
       scripCode  tickSize  lotSize      expiry  multiplier tradingSymbol
16        440938       1.0        5  28/11/2025         1.0       SILVERM
2176      450659       1.0        1  28/11/2025         1.0     SILVERMIC
2382      451669       1.0        5  27/02/2026         1.0       SILVERM
4557      457533       1.0        5  30/04/2026         1.0       SILVERM
4943      458305       1.0        1  27/02/2026         1.0     SILVERMIC
8652      463393       1.0      100  03/10/2025         0.1         GOLDM
9287      464151       1.0        5  30/06/2026         1.0       SILVERM
11026     466029       1.0        1  30/04/2026         1.0     SILVERMIC
11146     466161       1.0      100  05/11/2025         0.1         GOLDM
15211     470295       1.0      100  05/12/2025         0.1         GOLDM


In [38]:
import pandas as pd
import datetime as dt
from datetime import datetime
from SharekhanApi.sharekhanConnect import SharekhanConnect

# Auth
api_key = "GIaiFf8eFa1gOfdC2W4m6RUb9YOvVYIu"
with open('access.txt', 'r') as file:
    access_token = file.read().strip()
exchange = "MX"

# Initialize connection
sharekhan = SharekhanConnect(api_key, access_token=access_token)

try:
    order = sharekhan.master(exchange)
    if order:
        print("Data fetched successfully.")
    else:
        print("No data received.")
        exit()
except Exception as e:
    print("Error occurred:", e)
    exit()

# Convert to DataFrame
frame = order['data']
df = pd.DataFrame(frame)


# Filter GOLDM/SILVERM Futures
filtered_df = df[
    df['tradingSymbol'].str.contains('GOLDM', case=False, na=False) &
    (df['optionType'] == 'FUT')
]

# Parse expiry as datetime
filtered_df = filtered_df.copy()
filtered_df['expiry'] = pd.to_datetime(filtered_df['expiry'], dayfirst=True, errors='coerce')

# Filter only future expiries
today = pd.Timestamp.today().normalize()
future_contracts = filtered_df[filtered_df['expiry'] >= today]

# Find row with nearest expiry
nearest = future_contracts.sort_values('expiry').iloc[0]

# Extract scripcode
scripcode = nearest['scripCode']
symbol = nearest['tradingSymbol']
expiry = nearest['expiry'].strftime('%Y-%m-%d')
print(f"\n🔍 Nearest expiry contract: {symbol} (scripCode: {scripcode}, expiry: {expiry})")

# Fetch historical data
interval = "5minute"
history = sharekhan.historicaldata(exchange, str(scripcode), interval)
print(history)

# Display
hist_df = pd.DataFrame(history['data'])
# Step 1: Convert to datetime with explicit format
hist_df['tradeDate'] = pd.to_datetime(hist_df['tradeDate'], format="%d/%m/%Y", errors='coerce')
filtered_data = hist_df[hist_df['tradeDate'] == pd.to_datetime("2025-09-04")]
print("\n📊 Historical Data:")
print(filtered_data)
pd.DataFrame(filtered_data).to_csv("Historical_Data.csv", index=False)

Data fetched successfully.

🔍 Nearest expiry contract: GOLDM (scripCode: 463393, expiry: 2025-10-03)
{'status': 200, 'message': 'chart', 'timestamp': '2025-09-09T17:59:54+05:30', 'data': [{'open': 107391.0, 'high': 107490.0, 'low': 107029.0, 'close': 107149.0, 'qty': 1609, 'tradeTime': '09:04:59', 'tradeDate': '8/9/2025'}, {'open': 107150.0, 'high': 107188.0, 'low': 107141.0, 'close': 107144.0, 'qty': 496, 'tradeTime': '09:09:58', 'tradeDate': '8/9/2025'}, {'open': 107144.0, 'high': 107144.0, 'low': 107065.0, 'close': 107102.0, 'qty': 560, 'tradeTime': '09:14:56', 'tradeDate': '8/9/2025'}, {'open': 107102.0, 'high': 107136.0, 'low': 107073.0, 'close': 107128.0, 'qty': 378, 'tradeTime': '09:19:58', 'tradeDate': '8/9/2025'}, {'open': 107127.0, 'high': 107177.0, 'low': 107111.0, 'close': 107160.0, 'qty': 194, 'tradeTime': '09:24:57', 'tradeDate': '8/9/2025'}, {'open': 107151.0, 'high': 107182.0, 'low': 107123.0, 'close': 107129.0, 'qty': 295, 'tradeTime': '09:29:59', 'tradeDate': '8/9/202

In [39]:
print(hist_df['tradeDate'].dtype)
print(hist_df['tradeTime'].dtype)


datetime64[ns]
object
